# Magic State Injection Circuit for [[7,1,3]] Color Code

This implements the exact circuit from the figure that creates the logical observable qubit |ψ_L(θ,φ)⟩

In [2]:
from bloqade import squin
import bloqade.tsim

In [33]:
# Magic state injection circuit for [[7,1,3]] color code
# Exact gate sequence from the specification:
# T1: √Y† on q[0–5]
# T2: CNOT(1→{2,3,4,5})
# T3: CNOT(0→{2,4})
# T4: √Y on q[2,3,4,5]
# T5: CNOT(2→3), CNOT(4→5)
# T6: CNOT(6→{1,3,5})
# T7: √Y on q[1,2,4]

@squin.kernel
def logical_qubit(theta: float = 0.0, phi: float = 0.0):
    # Allocate 7 qubits
    q = squin.qalloc(7)
    
    # T1: √Y† on q[0–5]
    # √Y† = RY(-π/2) = -1.5707963267948966
    
    for i in [0, 1, 2, 3, 4, 5]:
        squin.ry(angle=-1.5707963267948966, qubit=q[i])
    
    # T2: CZ(1→2), CZ(3→4), CZ(5→6)
    for pair in [(1, 2), (3, 4), (5, 6)]:
        squin.cz(control=q[pair[0]], target=q[pair[1]])

    # T3: √Y on q[6]
    # √Y = RY(π/2) = 1.5707963267948966
    squin.ry(angle=1.5707963267948966, qubit=q[6])
    
    # T4: CZ(0→3), CZ(2→5), CZ(4→6)
    for pair in [(0, 3), (2, 5), (4, 6)]:
        squin.cz(control=q[pair[0]], target=q[pair[1]])

    # T5: √Y on q[2,3,4,5,6]
    for i in [2, 3, 4, 5, 6]:
        squin.ry(angle=1.5707963267948966, qubit=q[i])

    # T6: CZ(0→1), CZ(2→3), CZ(4→5)
    for pair in [(0, 1), (2, 3), (4, 5)]:
        squin.cz(control=q[pair[0]], target=q[pair[1]])
    
    # T7: √Y on q[1,2,4]
    for i in [1, 2, 4]:
        squin.ry(angle=1.5707963267948966, qubit=q[i])
    
# Visualize with white background
from IPython.display import HTML, display
tsim_circ = bloqade.tsim.Circuit(logical_qubit)
diagram = tsim_circ.diagram(height=500)
# Get HTML and wrap with white background
html_str = diagram._repr_html_() if hasattr(diagram, '_repr_html_') else str(diagram)
display(HTML(f'<div style="background-color: white; padding: 20px; border: 1px solid #ddd; display: inline-block;">{html_str}</div>'))

In [ ]:
# two copies of the logical qubit circuit: q[0-6], q[7-13]
# Gates are applied in parallel (same timestep for both logical qubits)
@squin.kernel
def _fixed():
    q = squin.qalloc(14)  # 3 registers × 7 qubits = 21 qubits
    
    # T1: √Y† on q[0–5], q[7–12], and q[14–19] (parallel)
    for i in [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12]:
        squin.ry(angle=-1.5707963267948966, qubit=q[i])  # √Y† = RY(-π/2)

    squin.h(q[13])
    
    # T2: CZ gates (parallel for both logical qubits)
    for pair in [(1, 2), (3, 4), (5, 6), (8, 9), (10, 11), (12, 13)]:
        squin.cz(control=q[pair[0]], target=q[pair[1]])
    
    # T3: √Y on q[6], q[13], and q[20] (parallel)
    squin.ry(angle=1.5707963267948966, qubit=q[6])
    squin.ry(angle=1.5707963267948966, qubit=q[13])
    
    # T4: CZ gates (parallel for all three logical qubits)
    for pair in [(0, 3), (2, 5), (4, 6), (7, 10), (9, 12), (11, 13)]:
        squin.cz(control=q[pair[0]], target=q[pair[1]])
    
    # T5: √Y on q[2,3,4,5,6], q[9,10,11,12,13], and q[16,17,18,19,20] (parallel)
    for i in [2, 3, 4, 5, 6, 9, 10, 11, 12, 13]:
        squin.ry(angle=1.5707963267948966, qubit=q[i])
    
    # T6: CZ gates (parallel for all three logical qubits)
    for pair in [(0, 1), (2, 3), (4, 5), (7, 8), (9, 10), (11, 12)]:
        squin.cz(control=q[pair[0]], target=q[pair[1]])
    
    # T7: √Y on q[1,2,4], q[8,9,11], and q[15,16,18] (parallel)
    for i in [1, 2, 4, 8, 9, 11]:
        squin.ry(angle=1.5707963267948966, qubit=q[i])

    for pair in [(0, 7), (1, 8), (2, 9), (3, 10), (4, 11), (5, 12), (6, 13)]:
        squin.cx(control=q[pair[0]], target=q[pair[1]])

    for i in range(7, 14):
        squin.measure(q[i])

    for i in range(7, 14):
        squin.reset(q[i])

# Visualize with white background
from IPython.display import HTML, display
tsim_circ = bloqade.tsim.Circuit(_fixed)
diagram = tsim_circ.diagram(height=500)
# Get HTML and wrap with white background
html_str = diagram._repr_html_() if hasattr(diagram, '_repr_html_') else str(diagram)
display(HTML(f'<div style="background-color: white; padding: 20px; border: 1px solid #ddd; display: inline-block;">{html_str}</div>'))

In [36]:
@squin.kernel
def _fixed():
    q = squin.qalloc(14)  # 3 registers × 7 qubits = 21 qubits
    
    # T1: √Y† on q[0–5], q[7–12], and q[14–19] (parallel)
    for i in [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12]:
        squin.ry(angle=-1.5707963267948966, qubit=q[i])  # √Y† = RY(-π/2)

    squin.h(q[13])
    # T2: CZ gates (parallel for all three logical qubits)
    for pair in [(1, 2), (3, 4), (5, 6), (8, 9), (10, 11), (12, 13)]:
        squin.cz(control=q[pair[0]], target=q[pair[1]])
    
    # T3: √Y on q[6], q[13], and q[20] (parallel)
    squin.ry(angle=1.5707963267948966, qubit=q[6])
    squin.ry(angle=1.5707963267948966, qubit=q[13])
    
    # T4: CZ gates (parallel for all three logical qubits)
    for pair in [(0, 3), (2, 5), (4, 6), (7, 10), (9, 12), (11, 13)]:
        squin.cz(control=q[pair[0]], target=q[pair[1]])
    
    # T5: √Y on q[2,3,4,5,6], q[9,10,11,12,13], and q[16,17,18,19,20] (parallel)
    for i in [2, 3, 4, 5, 6, 9, 10, 11, 12, 13]:
        squin.ry(angle=1.5707963267948966, qubit=q[i])
    
    # T6: CZ gates (parallel for all three logical qubits)
    for pair in [(0, 1), (2, 3), (4, 5), (7, 8), (9, 10), (11, 12)]:
        squin.cz(control=q[pair[0]], target=q[pair[1]])
    
    # T7: √Y on q[1,2,4], q[8,9,11], and q[15,16,18] (parallel)
    for i in [1, 2, 4, 8, 9, 11]:
        squin.ry(angle=1.5707963267948966, qubit=q[i])

    for pair in [(0, 7), (1, 8), (2, 9), (3, 10), (4, 11), (5, 12), (6, 13)]:
        squin.cx(control=q[pair[0]], target=q[pair[1]])

    for i in range(7, 14):
        squin.measure(q[i])

    for i in range(7, 14):
        squin.reset(q[i])

    for i in range(7, 13):
        squin.ry(angle=-1.5707963267948966, qubit=q[i])
    # T2: CZ gates (parallel for all three logical qubits)
    
    for pair in [(8, 9), (10, 11), (12, 13)]:
        squin.cz(control=q[pair[0]], target=q[pair[1]])
    
    # T3: √Y on q[6], q[13], and q[20] (parallel)
    squin.ry(angle=1.5707963267948966, qubit=q[13])
    
    # T4: CZ gates (parallel for all three logical qubits)
    for pair in [(7, 10), (9, 12), (11, 13)]:
        squin.cz(control=q[pair[0]], target=q[pair[1]])
    
    # T5: √Y on q[2,3,4,5,6], q[9,10,11,12,13], and q[16,17,18,19,20] (parallel)
    for i in [9, 10, 11, 12, 13]:
        squin.ry(angle=1.5707963267948966, qubit=q[i])
    
    # T6: CZ gates (parallel for all three logical qubits)
    for pair in [(7, 8), (9, 10), (11, 12)]:
        squin.cz(control=q[pair[0]], target=q[pair[1]])
    
    # T7: √Y on q[1,2,4], q[8,9,11], and q[15,16,18] (parallel)
    for i in [8, 9, 11]:
        squin.ry(angle=1.5707963267948966, qubit=q[i])

    for pair in [(0, 7), (1, 8), (2, 9), (3, 10), (4, 11), (5, 12), (6, 13)]:
        squin.cx(control=q[pair[0]], target=q[pair[1]])

    for i in range(7):
        squin.h(q[7 + i])

    for i in range(7):
        squin.measure(q[7 + i])

# Visualize with white background
from IPython.display import HTML, display
tsim_circ = bloqade.tsim.Circuit(_fixed)
diagram = tsim_circ.diagram(height=500)
# Get HTML and wrap with white background
html_str = diagram._repr_html_() if hasattr(diagram, '_repr_html_') else str(diagram)
display(HTML(f'<div style="background-color: white; padding: 20px; border: 1px solid #ddd; display: inline-block;">{html_str}</div>'))

In [37]:
from random import sample
import bloqade.stim

shot_length = 1000
stim_circ = bloqade.stim.Circuit(_fixed)
sampler = stim_circ.compile_sampler()
samples = sampler.sample(shots=shot_length)

In [38]:
def green_blue_orange_stabilizers(sample):
    z = (-1)**sample
    green = z[1] * z[2] * z[4] * z[5]
    blue = z[0] * z[1] * z[2] * z[3]
    orange = z[2] * z[3] * z[4] * z[6]
    return [green, blue, orange]

In [ ]:

first_z_stabilizer = green_blue_orange_stabilizers(samples[0][:7])
first_x_stabilizer = green_blue_orange_stabilizers(samples[0][7:])

count = 0
for i, sample in enumerate(samples):
    z_results = sample[:7]
    x_results = sample[7:]

    z_stabilizer = green_blue_orange_stabilizers(z_results)
    x_stabilizer = green_blue_orange_stabilizers(x_results)

    if z_stabilizer != first_z_stabilizer or x_stabilizer != first_x_stabilizer:
        print(f"error at {i}")
        count+=1
print(count)

0


In [10]:
ideal_z_stabilizer = first_z_stabilizer
ideal_x_stabilizer = first_x_stabilizer

In [11]:
from random import sample
import bloqade.stim

shot_length = 1000
stim_circ = bloqade.stim.Circuit(_fixed)
sampler = stim_circ.compile_sampler()
samples = sampler.sample(shots=shot_length)

In [12]:
samples[0]

array([False,  True, False,  True,  True, False, False,  True, False,
       False, False, False, False, False])

In [13]:


def green_blue_orange_stabilizers(sample):
    z = (-1)**sample
    green = z[1] * z[2] * z[4] * z[5]
    blue = z[0] * z[1] * z[2] * z[3]
    orange = z[2] * z[3] * z[4] * z[6]
    return [green, blue, orange]

print((-1)**samples[0])
print((-1)**samples[1])
print((-1)**samples[2])

print(green_blue_orange_stabilizers(samples[0]))
print(green_blue_orange_stabilizers(samples[1]))
print(green_blue_orange_stabilizers(samples[2]))

first_stabilizer = green_blue_orange_stabilizers(samples[0])
    
for i, sample in enumerate(samples): 
    stabilizer = green_blue_orange_stabilizers(sample)
    if stabilizer != first_stabilizer:
        print(f"error at {i}")

len(samples)

[ 1 -1  1 -1 -1  1  1 -1  1  1  1  1  1  1]
[ 1  1 -1 -1  1 -1  1 -1  1 -1 -1  1 -1  1]
[ 1 -1  1 -1  1 -1 -1  1 -1 -1 -1  1  1  1]
[np.int64(1), np.int64(1), np.int64(1)]
[np.int64(1), np.int64(1), np.int64(1)]
[np.int64(1), np.int64(1), np.int64(1)]


1000

In [27]:
from bloqade.cirq_utils import noise
from bloqade.cirq_utils.emit import emit_circuit
from bloqade.cirq_utils import load_circuit

cirq_main = emit_circuit(_fixed)
noise_model = (
    noise.GeminiOneZoneNoiseModel()
)
main_noisy = noise.transform_circuit(
    cirq_main, model=noise_model
)  # annotate circuit w noise
squin_main = load_circuit(main_noisy)  # back to Squin
noisy_stim = bloqade.tsim.Circuit(squin_main)

noise_scale = 1
noise_model_1 = noise.GeminiOneZoneNoiseModel(scaling_factor=noise_scale)

noisy1 = noise.transform_circuit(cirq_main, model=noise_model_1)
squin_noisy1 = load_circuit(noisy1)
noisy_stim1 = bloqade.tsim.Circuit(squin_noisy1)

# Example 2: changing just some of the channels: no local phase-z errors
noise_model_2 = noise.GeminiOneZoneNoiseModel(local_pz=0.0, global_pz=0.0)
noisy2 = noise.transform_circuit(cirq_main, model=noise_model_2)
squin_noisy2 = load_circuit(noisy2)
noisy_stim2 = bloqade.tsim.Circuit(squin_noisy2)

from IPython.display import HTML, display
diagram = noisy_stim2.diagram(height=600)
# Get HTML and wrap with white background
html_str = diagram._repr_html_() if hasattr(diagram, '_repr_html_') else str(diagram)
display(HTML(f'<div style="background-color: white; padding: 20px; border: 1px solid #ddd; display: inline-block;">{html_str}</div>'))

In [28]:
from random import sample
import bloqade.stim

shot_length = 1000
# stim_circ = bloqade.stim.Circuit(_fixed)
sampler = noisy_stim2.compile_sampler()
samples = sampler.sample(shots=shot_length)
samples[0]

array([False,  True, False, False,  True,  True,  True,  True, False,
       False,  True,  True,  True,  True])

In [29]:
# compare to the ideal case (diagram without noise)
z_error = 0
x_error = 0
idx_zerror = []
idx_xerror = []
index_list = [i for i in range(shot_length)] # 0..999
print(len(index_list))

for i, sample in enumerate(samples):
    z_results = sample[:7]
    x_results = sample[:7]

    z_stabilizer = green_blue_orange_stabilizers(z_results)
    x_stabilizer = green_blue_orange_stabilizers(x_results)
    
    if z_stabilizer != ideal_z_stabilizer:
        z_error += 1
        print(f"Z error at {i}")
        idx_zerror.append(i)
        index_list.remove(i)

    if x_stabilizer != ideal_x_stabilizer:
        x_error += 1
        print(f"X error at {i}")
        idx_xerror.append(i)
        if i in index_list: 
            index_list.remove(i)

print("Z error count: ", z_error)
print("X error count: ", x_error)
print(index_list)

1000
Z error at 0
X error at 0
X error at 1
X error at 2
X error at 3
Z error at 4
X error at 4
X error at 5
Z error at 6
X error at 6
Z error at 7
X error at 7
Z error at 8
X error at 8
X error at 9
X error at 10
Z error at 11
X error at 11
Z error at 12
X error at 12
Z error at 13
Z error at 14
X error at 14
Z error at 15
X error at 15
X error at 16
X error at 17
Z error at 18
X error at 18
Z error at 19
X error at 19
X error at 20
X error at 21
Z error at 22
X error at 22
Z error at 23
X error at 23
Z error at 24
X error at 24
Z error at 25
X error at 25
X error at 26
X error at 27
X error at 28
Z error at 29
X error at 29
Z error at 30
X error at 30
X error at 31
X error at 32
X error at 33
Z error at 34
X error at 34
Z error at 35
X error at 35
X error at 36
Z error at 37
X error at 37
X error at 38
Z error at 39
X error at 39
Z error at 40
X error at 41
Z error at 42
X error at 42
X error at 43
X error at 44
X error at 45
Z error at 46
X error at 46
Z error at 47
Z error at 48
X 

In [17]:
# compare to the ideal case (diagram without noise)
count = 0
for i, sample in enumerate(samples):
    x_results = sample[7:]

    x_stabilizer = green_blue_orange_stabilizers(x_results)

    if x_stabilizer != ideal_x_stabilizer:
        count += 1
        print(f"error at {i}")

print(count)

error at 1
error at 6
error at 7
error at 12
error at 13
error at 17
error at 23
error at 29
error at 32
error at 44
error at 46
error at 50
error at 69
error at 77
error at 80
error at 84
error at 87
error at 90
error at 94
error at 96
error at 98
error at 100
error at 102
error at 109
error at 120
error at 122
error at 125
error at 129
error at 137
error at 144
error at 145
error at 147
error at 159
error at 160
error at 162
error at 166
error at 168
error at 174
error at 182
error at 183
error at 184
error at 199
error at 200
error at 214
error at 215
error at 221
error at 223
error at 229
error at 233
error at 235
error at 238
error at 239
error at 242
error at 251
error at 253
error at 261
error at 263
error at 270
error at 274
error at 276
error at 278
error at 279
error at 286
error at 287
error at 288
error at 290
error at 294
error at 297
error at 305
error at 311
error at 322
error at 326
error at 327
error at 328
error at 332
error at 333
error at 343
error at 350
error at 3

In [48]:
@squin.kernel
def _five_layer_fixed():
    ## Cycle 1

    q = squin.qalloc(14)  # 3 registers × 7 qubits = 21 qubits
    
    # T1: √Y† on q[0–5], q[7–12], and q[14–19] (parallel)
    for i in [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12]:
        squin.ry(angle=-1.5707963267948966, qubit=q[i])  # √Y† = RY(-π/2)

    squin.h(q[13])
    # T2: CZ gates (parallel for all three logical qubits)
    for pair in [(1, 2), (3, 4), (5, 6), (8, 9), (10, 11), (12, 13)]:
        squin.cz(control=q[pair[0]], target=q[pair[1]])
    
    # T3: √Y on q[6], q[13], and q[20] (parallel)
    squin.ry(angle=1.5707963267948966, qubit=q[6])
    squin.ry(angle=1.5707963267948966, qubit=q[13])
    
    # T4: CZ gates (parallel for all three logical qubits)
    for pair in [(0, 3), (2, 5), (4, 6), (7, 10), (9, 12), (11, 13)]:
        squin.cz(control=q[pair[0]], target=q[pair[1]])
    
    # T5: √Y on q[2,3,4,5,6], q[9,10,11,12,13], and q[16,17,18,19,20] (parallel)
    for i in [2, 3, 4, 5, 6, 9, 10, 11, 12, 13]:
        squin.ry(angle=1.5707963267948966, qubit=q[i])
    
    # T6: CZ gates (parallel for all three logical qubits)
    for pair in [(0, 1), (2, 3), (4, 5), (7, 8), (9, 10), (11, 12)]:
        squin.cz(control=q[pair[0]], target=q[pair[1]])
    
    # T7: √Y on q[1,2,4], q[8,9,11], and q[15,16,18] (parallel)
    for i in [1, 2, 4, 8, 9, 11]:
        squin.ry(angle=1.5707963267948966, qubit=q[i])

    for pair in [(0, 7), (1, 8), (2, 9), (3, 10), (4, 11), (5, 12), (6, 13)]:
        squin.cx(control=q[pair[0]], target=q[pair[1]])

    for i in range(7, 14):
        squin.measure(q[i])

    for i in range(7, 14):
        squin.reset(q[i])

    for i in range(7, 13):
        squin.ry(angle=-1.5707963267948966, qubit=q[i])
    # T2: CZ gates (parallel for all three logical qubits)
    
    for pair in [(8, 9), (10, 11), (12, 13)]:
        squin.cz(control=q[pair[0]], target=q[pair[1]])
    
    # T3: √Y on q[6], q[13], and q[20] (parallel)
    squin.ry(angle=1.5707963267948966, qubit=q[13])
    
    # T4: CZ gates (parallel for all three logical qubits)
    for pair in [(7, 10), (9, 12), (11, 13)]:
        squin.cz(control=q[pair[0]], target=q[pair[1]])
    
    # T5: √Y on q[2,3,4,5,6], q[9,10,11,12,13], and q[16,17,18,19,20] (parallel)
    for i in [9, 10, 11, 12, 13]:
        squin.ry(angle=1.5707963267948966, qubit=q[i])
    
    # T6: CZ gates (parallel for all three logical qubits)
    for pair in [(7, 8), (9, 10), (11, 12)]:
        squin.cz(control=q[pair[0]], target=q[pair[1]])
    
    # T7: √Y on q[1,2,4], q[8,9,11], and q[15,16,18] (parallel)
    for i in [8, 9, 11]:
        squin.ry(angle=1.5707963267948966, qubit=q[i])

    for pair in [(0, 7), (1, 8), (2, 9), (3, 10), (4, 11), (5, 12), (6, 13)]:
        squin.cx(control=q[pair[0]], target=q[pair[1]])

    for i in range(7):
        squin.h(q[7 + i])

    for i in range(7):
        squin.measure(q[7 + i])


    ## Cycle 2

    for i in range(7, 14):
        squin.reset(q[i])
    
    # T1: √Y† on q[0–5], q[7–12], and q[14–19] (parallel)
    for i in [7, 8, 9, 10, 11, 12]:
        squin.ry(angle=-1.5707963267948966, qubit=q[i])  # √Y† = RY(-π/2)

    squin.h(q[13])
    # T2: CZ gates (parallel for all three logical qubits)
    for pair in [(8, 9), (10, 11), (12, 13)]:
        squin.cz(control=q[pair[0]], target=q[pair[1]])
    
    # T3: √Y on q[6], q[13], and q[20] (parallel)
    squin.ry(angle=1.5707963267948966, qubit=q[13])
    
    # T4: CZ gates (parallel for all three logical qubits)
    for pair in [(7, 10), (9, 12), (11, 13)]:
        squin.cz(control=q[pair[0]], target=q[pair[1]])
    
    # T5: √Y on q[2,3,4,5,6], q[9,10,11,12,13], and q[16,17,18,19,20] (parallel)
    for i in [9, 10, 11, 12, 13]:
        squin.ry(angle=1.5707963267948966, qubit=q[i])
    
    # T6: CZ gates (parallel for all three logical qubits)
    for pair in [(7, 8), (9, 10), (11, 12)]:
        squin.cz(control=q[pair[0]], target=q[pair[1]])
    
    # T7: √Y on q[1,2,4], q[8,9,11], and q[15,16,18] (parallel)
    for i in [8, 9, 11]:
        squin.ry(angle=1.5707963267948966, qubit=q[i])

    for pair in [(0, 7), (1, 8), (2, 9), (3, 10), (4, 11), (5, 12), (6, 13)]:
        squin.cx(control=q[pair[0]], target=q[pair[1]])

    for i in range(7, 14):
        squin.measure(q[i])

    for i in range(7, 14):
        squin.reset(q[i])

    for i in range(7, 13):
        squin.ry(angle=-1.5707963267948966, qubit=q[i])
    # T2: CZ gates (parallel for all three logical qubits)
    
    for pair in [(8, 9), (10, 11), (12, 13)]:
        squin.cz(control=q[pair[0]], target=q[pair[1]])
    
    # T3: √Y on q[6], q[13], and q[20] (parallel)
    squin.ry(angle=1.5707963267948966, qubit=q[13])
    
    # T4: CZ gates (parallel for all three logical qubits)
    for pair in [(7, 10), (9, 12), (11, 13)]:
        squin.cz(control=q[pair[0]], target=q[pair[1]])
    
    # T5: √Y on q[2,3,4,5,6], q[9,10,11,12,13], and q[16,17,18,19,20] (parallel)
    for i in [9, 10, 11, 12, 13]:
        squin.ry(angle=1.5707963267948966, qubit=q[i])
    
    # T6: CZ gates (parallel for all three logical qubits)
    for pair in [(7, 8), (9, 10), (11, 12)]:
        squin.cz(control=q[pair[0]], target=q[pair[1]])
    
    # T7: √Y on q[1,2,4], q[8,9,11], and q[15,16,18] (parallel)
    for i in [8, 9, 11]:
        squin.ry(angle=1.5707963267948966, qubit=q[i])

    for pair in [(0, 7), (1, 8), (2, 9), (3, 10), (4, 11), (5, 12), (6, 13)]:
        squin.cx(control=q[pair[0]], target=q[pair[1]])

    for i in range(7):
        squin.h(q[7 + i])

    for i in range(7):
        squin.measure(q[7 + i])


    ## Cycle 3

    for i in range(7, 14):
        squin.reset(q[i])
    
    # T1: √Y† on q[0–5], q[7–12], and q[14–19] (parallel)
    for i in [7, 8, 9, 10, 11, 12]:
        squin.ry(angle=-1.5707963267948966, qubit=q[i])  # √Y† = RY(-π/2)

    squin.h(q[13])
    # T2: CZ gates (parallel for all three logical qubits)
    for pair in [(8, 9), (10, 11), (12, 13)]:
        squin.cz(control=q[pair[0]], target=q[pair[1]])
    
    # T3: √Y on q[6], q[13], and q[20] (parallel)
    squin.ry(angle=1.5707963267948966, qubit=q[13])
    
    # T4: CZ gates (parallel for all three logical qubits)
    for pair in [(7, 10), (9, 12), (11, 13)]:
        squin.cz(control=q[pair[0]], target=q[pair[1]])
    
    # T5: √Y on q[2,3,4,5,6], q[9,10,11,12,13], and q[16,17,18,19,20] (parallel)
    for i in [9, 10, 11, 12, 13]:
        squin.ry(angle=1.5707963267948966, qubit=q[i])
    
    # T6: CZ gates (parallel for all three logical qubits)
    for pair in [(7, 8), (9, 10), (11, 12)]:
        squin.cz(control=q[pair[0]], target=q[pair[1]])
    
    # T7: √Y on q[1,2,4], q[8,9,11], and q[15,16,18] (parallel)
    for i in [8, 9, 11]:
        squin.ry(angle=1.5707963267948966, qubit=q[i])

    for pair in [(0, 7), (1, 8), (2, 9), (3, 10), (4, 11), (5, 12), (6, 13)]:
        squin.cx(control=q[pair[0]], target=q[pair[1]])

    for i in range(7, 14):
        squin.measure(q[i])

    for i in range(7, 14):
        squin.reset(q[i])

    for i in range(7, 13):
        squin.ry(angle=-1.5707963267948966, qubit=q[i])
    # T2: CZ gates (parallel for all three logical qubits)
    
    for pair in [(8, 9), (10, 11), (12, 13)]:
        squin.cz(control=q[pair[0]], target=q[pair[1]])
    
    # T3: √Y on q[6], q[13], and q[20] (parallel)
    squin.ry(angle=1.5707963267948966, qubit=q[13])
    
    # T4: CZ gates (parallel for all three logical qubits)
    for pair in [(7, 10), (9, 12), (11, 13)]:
        squin.cz(control=q[pair[0]], target=q[pair[1]])
    
    # T5: √Y on q[2,3,4,5,6], q[9,10,11,12,13], and q[16,17,18,19,20] (parallel)
    for i in [9, 10, 11, 12, 13]:
        squin.ry(angle=1.5707963267948966, qubit=q[i])
    
    # T6: CZ gates (parallel for all three logical qubits)
    for pair in [(7, 8), (9, 10), (11, 12)]:
        squin.cz(control=q[pair[0]], target=q[pair[1]])
    
    # T7: √Y on q[1,2,4], q[8,9,11], and q[15,16,18] (parallel)
    for i in [8, 9, 11]:
        squin.ry(angle=1.5707963267948966, qubit=q[i])

    for pair in [(0, 7), (1, 8), (2, 9), (3, 10), (4, 11), (5, 12), (6, 13)]:
        squin.cx(control=q[pair[0]], target=q[pair[1]])

    for i in range(7):
        squin.h(q[7 + i])

    for i in range(7):
        squin.measure(q[7 + i])

    
    ## Cycle 4

    for i in range(7, 14):
        squin.reset(q[i])
    
    # T1: √Y† on q[0–5], q[7–12], and q[14–19] (parallel)
    for i in [7, 8, 9, 10, 11, 12]:
        squin.ry(angle=-1.5707963267948966, qubit=q[i])  # √Y† = RY(-π/2)

    squin.h(q[13])
    # T2: CZ gates (parallel for all three logical qubits)
    for pair in [(8, 9), (10, 11), (12, 13)]:
        squin.cz(control=q[pair[0]], target=q[pair[1]])
    
    # T3: √Y on q[6], q[13], and q[20] (parallel)
    squin.ry(angle=1.5707963267948966, qubit=q[13])
    
    # T4: CZ gates (parallel for all three logical qubits)
    for pair in [(7, 10), (9, 12), (11, 13)]:
        squin.cz(control=q[pair[0]], target=q[pair[1]])
    
    # T5: √Y on q[2,3,4,5,6], q[9,10,11,12,13], and q[16,17,18,19,20] (parallel)
    for i in [9, 10, 11, 12, 13]:
        squin.ry(angle=1.5707963267948966, qubit=q[i])
    
    # T6: CZ gates (parallel for all three logical qubits)
    for pair in [(7, 8), (9, 10), (11, 12)]:
        squin.cz(control=q[pair[0]], target=q[pair[1]])
    
    # T7: √Y on q[1,2,4], q[8,9,11], and q[15,16,18] (parallel)
    for i in [8, 9, 11]:
        squin.ry(angle=1.5707963267948966, qubit=q[i])

    for pair in [(0, 7), (1, 8), (2, 9), (3, 10), (4, 11), (5, 12), (6, 13)]:
        squin.cx(control=q[pair[0]], target=q[pair[1]])

    for i in range(7, 14):
        squin.measure(q[i])

    for i in range(7, 14):
        squin.reset(q[i])

    for i in range(7, 13):
        squin.ry(angle=-1.5707963267948966, qubit=q[i])
    # T2: CZ gates (parallel for all three logical qubits)
    
    for pair in [(8, 9), (10, 11), (12, 13)]:
        squin.cz(control=q[pair[0]], target=q[pair[1]])
    
    # T3: √Y on q[6], q[13], and q[20] (parallel)
    squin.ry(angle=1.5707963267948966, qubit=q[13])
    
    # T4: CZ gates (parallel for all three logical qubits)
    for pair in [(7, 10), (9, 12), (11, 13)]:
        squin.cz(control=q[pair[0]], target=q[pair[1]])
    
    # T5: √Y on q[2,3,4,5,6], q[9,10,11,12,13], and q[16,17,18,19,20] (parallel)
    for i in [9, 10, 11, 12, 13]:
        squin.ry(angle=1.5707963267948966, qubit=q[i])
    
    # T6: CZ gates (parallel for all three logical qubits)
    for pair in [(7, 8), (9, 10), (11, 12)]:
        squin.cz(control=q[pair[0]], target=q[pair[1]])
    
    # T7: √Y on q[1,2,4], q[8,9,11], and q[15,16,18] (parallel)
    for i in [8, 9, 11]:
        squin.ry(angle=1.5707963267948966, qubit=q[i])

    for pair in [(0, 7), (1, 8), (2, 9), (3, 10), (4, 11), (5, 12), (6, 13)]:
        squin.cx(control=q[pair[0]], target=q[pair[1]])

    for i in range(7):
        squin.h(q[7 + i])

    for i in range(7):
        squin.measure(q[7 + i])


    ## Cycle 5

    for i in range(7, 14):
        squin.reset(q[i])
    
    # T1: √Y† on q[0–5], q[7–12], and q[14–19] (parallel)
    for i in [7, 8, 9, 10, 11, 12]:
        squin.ry(angle=-1.5707963267948966, qubit=q[i])  # √Y† = RY(-π/2)

    squin.h(q[13])
    # T2: CZ gates (parallel for all three logical qubits)
    for pair in [(8, 9), (10, 11), (12, 13)]:
        squin.cz(control=q[pair[0]], target=q[pair[1]])
    
    # T3: √Y on q[6], q[13], and q[20] (parallel)
    squin.ry(angle=1.5707963267948966, qubit=q[13])
    
    # T4: CZ gates (parallel for all three logical qubits)
    for pair in [(7, 10), (9, 12), (11, 13)]:
        squin.cz(control=q[pair[0]], target=q[pair[1]])
    
    # T5: √Y on q[2,3,4,5,6], q[9,10,11,12,13], and q[16,17,18,19,20] (parallel)
    for i in [9, 10, 11, 12, 13]:
        squin.ry(angle=1.5707963267948966, qubit=q[i])
    
    # T6: CZ gates (parallel for all three logical qubits)
    for pair in [(7, 8), (9, 10), (11, 12)]:
        squin.cz(control=q[pair[0]], target=q[pair[1]])
    
    # T7: √Y on q[1,2,4], q[8,9,11], and q[15,16,18] (parallel)
    for i in [8, 9, 11]:
        squin.ry(angle=1.5707963267948966, qubit=q[i])

    for pair in [(0, 7), (1, 8), (2, 9), (3, 10), (4, 11), (5, 12), (6, 13)]:
        squin.cx(control=q[pair[0]], target=q[pair[1]])

    for i in range(7, 14):
        squin.measure(q[i])

    for i in range(7, 14):
        squin.reset(q[i])

    for i in range(7, 13):
        squin.ry(angle=-1.5707963267948966, qubit=q[i])
    # T2: CZ gates (parallel for all three logical qubits)
    
    for pair in [(8, 9), (10, 11), (12, 13)]:
        squin.cz(control=q[pair[0]], target=q[pair[1]])
    
    # T3: √Y on q[6], q[13], and q[20] (parallel)
    squin.ry(angle=1.5707963267948966, qubit=q[13])
    
    # T4: CZ gates (parallel for all three logical qubits)
    for pair in [(7, 10), (9, 12), (11, 13)]:
        squin.cz(control=q[pair[0]], target=q[pair[1]])
    
    # T5: √Y on q[2,3,4,5,6], q[9,10,11,12,13], and q[16,17,18,19,20] (parallel)
    for i in [9, 10, 11, 12, 13]:
        squin.ry(angle=1.5707963267948966, qubit=q[i])
    
    # T6: CZ gates (parallel for all three logical qubits)
    for pair in [(7, 8), (9, 10), (11, 12)]:
        squin.cz(control=q[pair[0]], target=q[pair[1]])
    
    # T7: √Y on q[1,2,4], q[8,9,11], and q[15,16,18] (parallel)
    for i in [8, 9, 11]:
        squin.ry(angle=1.5707963267948966, qubit=q[i])

    for pair in [(0, 7), (1, 8), (2, 9), (3, 10), (4, 11), (5, 12), (6, 13)]:
        squin.cx(control=q[pair[0]], target=q[pair[1]])

    for i in range(7):
        squin.h(q[7 + i])

    for i in range(7):
        squin.measure(q[7 + i])


# Visualize with white background
from IPython.display import HTML, display
tsim_circ = bloqade.tsim.Circuit(_fixed)
diagram = tsim_circ.diagram(height=500)
# Get HTML and wrap with white background
html_str = diagram._repr_html_() if hasattr(diagram, '_repr_html_') else str(diagram)
display(HTML(f'<div style="background-color: white; padding: 20px; border: 1px solid #ddd; display: inline-block;">{html_str}</div>'))

In [49]:
from random import sample
import bloqade.stim

shot_length = 1000
stim_circ = bloqade.stim.Circuit(_five_layer_fixed)
sampler = stim_circ.compile_sampler()
samples = sampler.sample(shots=shot_length)

In [50]:
from bloqade.cirq_utils import noise
from bloqade.cirq_utils.emit import emit_circuit
from bloqade.cirq_utils import load_circuit

cirq_main = emit_circuit(_five_layer_fixed)
noise_model = (
    noise.GeminiOneZoneNoiseModel()
)
main_noisy = noise.transform_circuit(
    cirq_main, model=noise_model
)  # annotate circuit w noise
squin_main = load_circuit(main_noisy)  # back to Squin
noisy_stim = bloqade.tsim.Circuit(squin_main)

noise_scale = 1
noise_model_1 = noise.GeminiOneZoneNoiseModel(scaling_factor=noise_scale)

noisy1 = noise.transform_circuit(cirq_main, model=noise_model_1)
squin_noisy1 = load_circuit(noisy1)
noisy_stim1 = bloqade.tsim.Circuit(squin_noisy1)

# Example 2: changing just some of the channels: no local phase-z errors
noise_model_2 = noise.GeminiOneZoneNoiseModel(local_pz=0.0, global_pz=0.0)
noisy2 = noise.transform_circuit(cirq_main, model=noise_model_2)
squin_noisy2 = load_circuit(noisy2)
noisy_stim2 = bloqade.tsim.Circuit(squin_noisy2)

from IPython.display import HTML, display
diagram = noisy_stim2.diagram(height=600)
# Get HTML and wrap with white background
html_str = diagram._repr_html_() if hasattr(diagram, '_repr_html_') else str(diagram)
display(HTML(f'<div style="background-color: white; padding: 20px; border: 1px solid #ddd; display: inline-block;">{html_str}</div>'))

In [ ]:
print("hello")